I want to convey two things in this notebook.
## 1. Don't have to be hesitant about using Loop.
They say "avoid loops!'.
But I think It's not bad idea to use loops for this competition.
Because:
* We have to use small batch inference using Time-series API.
* Loops have very small overhead for each batch.
* Loops are more flexible.
* Even loops are not so slow. 3 features are extracted within 10 minits for 100M train data, as you can see blow.

## 2. Future information should not be used.
Time-series API doesn't allow us to use information from the future.
So we should not use it, especially user statistics from future make things very bad.

In [1]:
import pickle
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict, deque
from tqdm.notebook import tqdm
import lightgbm as lgb

## setting
CV files are generated by [this notebook](https://www.kaggle.com/its7171/cv-strategy)

In [2]:
train_pickle = '../input/riiid-cross-validation-files/cv1_train.pickle'
valid_pickle = '../input/riiid-cross-validation-files/cv1_valid.pickle'
question_file = '../input/features/question3.csv'
debug = False
validaten_flg = False

## feature engineering

In [3]:
import pickle

loaded_pickle = open("../input/for-transformer/train_current_part.pkl", "rb")
train_current_part = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_current_question.pkl", "rb")
train_current_question = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_current_tag.pkl", "rb")
train_current_tag = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_other_feats.pkl", "rb")
train_other_feats = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_past_part_answ.pkl", "rb")
train_past_part_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_past_quest_answ.pkl", "rb")
train_past_quest_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_past_tag_answ.pkl", "rb")
train_past_tag_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/train_y.pkl", "rb")
train_y = pickle.load(loaded_pickle)


loaded_pickle = open("../input/for-transformer/valid_current_part.pkl", "rb")
valid_current_part = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_current_question.pkl", "rb")
valid_current_question = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_current_tag.pkl", "rb")
valid_current_tag = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_other_feats.pkl", "rb")
valid_other_feats = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_past_part_answ.pkl", "rb")
valid_past_part_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_past_quest_answ.pkl", "rb")
valid_past_quest_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_past_tag_answ.pkl", "rb")
valid_past_tag_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-transformer/valid_y.pkl", "rb")
valid_y = pickle.load(loaded_pickle)



In [4]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM, Masking, Embedding, Concatenate, Input, Reshape,Flatten, AveragePooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.metrics import AUC
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda
#from tensorflow.keras.layers import merge
from tensorflow.keras.layers import multiply, Reshape
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm import tqdm
from tqdm import trange
from tensorflow.keras.utils import Sequence

## modeling

In [5]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)




def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)




def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)




def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
      q: query shape == (..., seq_len_q, depth)
      k: key shape == (..., seq_len_k, depth)
      v: value shape == (..., seq_len_v, depth_v)
      mask: Float tensor with shape broadcastable 
            to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
      output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights




class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                       (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights





def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])


    
    
    
class EncoderLayer2(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.2):
        super(EncoderLayer2, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2



class Encoder2(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
                   maximum_position_encoding, rate=0.2):
        super(Encoder2, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        #self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                                self.d_model)


        self.enc_layers = [EncoderLayer2(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
        #x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

    
class DecoderLayer2(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.2):
        super(DecoderLayer2, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)


    def call(self, x, enc_output, training, 
                look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(
                enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

        return out3, attn_weights_block1, attn_weights_block2

    
    
class Decoder2(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
                    maximum_position_encoding, rate=0.2):
        super(Decoder2, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        #self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer2(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, 
               look_ahead_mask, padding_mask):

        seq_len = tf.shape(x)[1]
        attention_weights = {}

        #x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
              x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

        attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
        attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights    

class Transformer2(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, padding_length, rate=0.2):
        super(Transformer2, self).__init__()

        self.encoder = Encoder2(num_layers, d_model, num_heads, dff, padding_length)
        
        self.decoder = Decoder2(num_layers, d_model, num_heads, dff, padding_length)

        self.second_final_layer = tf.keras.layers.Dense(dff)
        self.final_layer = Dense(1,activation = 'sigmoid')
    
    def call(self, inp1, inp2, training, en_combined_mask, de_look_ahead_mask, de_padding_mask):

        enc_output = self.encoder(inp1, training, en_combined_mask)  # (batch_size, inp_seq_len, d_model)
        dec_output, attention_weights = self.decoder(
                inp2, enc_output, training, de_look_ahead_mask, de_padding_mask)
            
        second_final_output = self.second_final_layer(dec_output)  # (batch_size, tar_seq_len, question_answer_pair_size)
        final_output = self.final_layer(second_final_output)
        return final_output



In [6]:
train_other_feats1 = np.reshape(train_other_feats[:,:,0],(-1,60,1))
train_other_feats2 = np.reshape(train_other_feats[:,:,1],(-1,60,1))

valid_other_feats1 = np.reshape(valid_other_feats[:,:,0],(-1,60,1))
valid_other_feats2 = np.reshape(valid_other_feats[:,:,1],(-1,60,1))

In [7]:

num_layers = 2
d_model = 128
num_heads = 2
dff = 512

question_answer_pair_size = 27048
tag_answer_pair_size = 3042
part_answer_pair_size = 16
n_question = 13524
n_tag = 1521
n_part = 8

pe_input = 60
epoch = 10
max_len = 100


def build(num_layers, d_model, num_heads, dff, n_question, n_tag, n_part, pe_input,
          question_answer_pair_size, tag_answer_pair_size, part_answer_pair_size):

    en_input1 = Input(batch_shape = (None, None), name = 'current_question')
    en_input1_embed = Embedding(n_question, d_model)(en_input1)
    en_input2 = Input(batch_shape = (None, None), name = 'current_tag')
    en_input2_embed = Embedding(n_tag, d_model)(en_input2)
    en_input3 = Input(batch_shape = (None, None), name = 'current_part')
    en_input3_embed = Embedding(n_part, d_model)(en_input3)
    en_input = tf.math.add_n([en_input1_embed, en_input2_embed, en_input3_embed])

    en_look_ahead_mask = create_look_ahead_mask(tf.shape(en_input1)[1])
    en_padding_mask = create_padding_mask(en_input1)
    en_combined_mask = tf.maximum(en_look_ahead_mask, en_padding_mask)
    
    
    
    #en_input1_embed = K.sum(en_input1_embed, axis = -2)
    de_input4 = Input(batch_shape = (None, None), name = 'past_question_answer')
    de_input4_embed = Embedding(question_answer_pair_size, d_model)(de_input4)
    de_input5 = Input(batch_shape = (None, None), name = 'past_tag_answer')
    de_input5_embed = Embedding(tag_answer_pair_size, d_model)(de_input5)
    de_input6 = Input(batch_shape = (None, None), name = 'past_part_answer')
    de_input6_embed = Embedding(part_answer_pair_size, d_model)(de_input6)
    de_input7 = Input(batch_shape = (None, None, 1), name = 'other_feature1')
    de_input7_masked = (Masking(mask_value= 0, input_shape = (None, None, 1)))(de_input7)
    de_input7_embed = Dense(d_model, input_shape = (None, None, 1))(de_input7_masked)
    de_input8 = Input(batch_shape = (None, None, 1), name = 'other_feature2')
    de_input8_masked = (Masking(mask_value= 0, input_shape = (None, None, 1)))(de_input8)
    de_input8_embed = Dense(d_model, input_shape = (None, None, 1))(de_input8_masked)   
    de_input = tf.math.add_n([de_input4_embed, de_input5_embed, de_input6_embed, de_input7_embed, de_input8_embed])
    
    de_look_ahead_mask = create_look_ahead_mask(tf.shape(de_input4)[1])
    de_padding_mask = create_padding_mask(de_input4)
    de_combined_mask = tf.maximum(de_look_ahead_mask, de_padding_mask)
    
    
    transformer = Transformer2(num_layers, d_model, num_heads, dff, pe_input)
    
    final_output = transformer(en_input, de_input, True, en_combined_mask, de_combined_mask, de_combined_mask)
    
    
    model = Model(inputs=[en_input1, en_input2, en_input3, de_input4, de_input5, de_input6, de_input7, de_input8], outputs=final_output)
    model.compile( optimizer = 'adam',
                    loss = 'binary_crossentropy',
                    metrics=['accuracy',AUC()])
    
    return model

#my_model = build(num_layers, d_model, num_heads, dff, n_question, n_tag, n_part, n_answer, pe_input, num_other_feats, max_len)


my_model = build(num_layers, d_model, num_heads, dff, n_question, n_tag, n_part, pe_input,
          question_answer_pair_size, tag_answer_pair_size, part_answer_pair_size)



In [8]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor=AUC(), patience=2,mode='max') 


my_model.fit([train_current_question, train_current_tag, train_current_part, 
              train_past_quest_answ, train_past_tag_answ, train_past_part_answ,
                               train_other_feats1, train_other_feats2],train_y, 
             validation_data=([valid_current_question, valid_current_tag, valid_current_part,
                               valid_past_quest_answ, valid_past_tag_answ, valid_past_part_answ,
                               valid_other_feats1, valid_other_feats2],valid_y), batch_size = 200,
             epochs = 15, verbose = 1, callbacks = [earlyStopping])

Epoch 1/15
2666/2666 [==============================] - 329s 124ms/step - loss: 0.5494 - accuracy: 0.7128 - auc: 0.7360 - val_loss: 0.4768 - val_accuracy: 0.7543 - val_auc: 0.8287
Epoch 2/15
2666/2666 [==============================] - 331s 124ms/step - loss: 0.5036 - accuracy: 0.7429 - auc: 0.7999 - val_loss: 0.4648 - val_accuracy: 0.7622 - val_auc: 0.8403
Epoch 3/15
2666/2666 [==============================] - 334s 125ms/step - loss: 0.4968 - accuracy: 0.7471 - auc: 0.8074 - val_loss: 0.4612 - val_accuracy: 0.7643 - val_auc: 0.8434
Epoch 4/15
2666/2666 [==============================] - 329s 123ms/step - loss: 0.4925 - accuracy: 0.7495 - auc: 0.8119 - val_loss: 0.4584 - val_accuracy: 0.7660 - val_auc: 0.8462
Epoch 5/15
2666/2666 [==============================] - 325s 122ms/step - loss: 0.4896 - accuracy: 0.7513 - auc: 0.8151 - val_loss: 0.4570 - val_accuracy: 0.7669 - val_auc: 0.8477
Epoch 6/15
2666/2666 [==============================] - 329s 123ms/step - loss: 0.4875 - accuracy: 0

In [9]:
my_model.save_weights('SAINT_model_feature_extraction.h5')

Have a fun with loops! :)

In [10]:
my_model.load_weights('../input/transformer-model/SAINT_model_feature_extraction.h5')

OSError: Unable to open file (unable to open file: name = '../input/transformer-model/SAINT_model_feature_extraction.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [11]:
y_hat = my_model.predict([valid_current_question, valid_current_tag, valid_current_part,
                               valid_past_quest_answ, valid_past_tag_answ, valid_past_part_answ,
                               valid_other_feats1, valid_other_feats2])

In [12]:
y_hat = y_hat[:,-1,0]
y = valid_y[:,-1,0]
print(roc_auc_score(y,y_hat))

0.7775014872403258
